In [21]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import scipy.stats as stats
import os
import pickle
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression

In [5]:
account_t=pd.read_csv('../accounts_data_trim.csv')
account_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64243 entries, 0 to 64242
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       64243 non-null  int64  
 1   Unnamed: 0.1     64243 non-null  int64  
 2   posts            64243 non-null  int64  
 3   flw              64243 non-null  int64  
 4   flg              64243 non-null  int64  
 5   pic              64243 non-null  int64  
 6   link             64243 non-null  int64  
 7   caption_len_avg  64243 non-null  int64  
 8   cap_zero_per     64243 non-null  float64
 9   no_image_per     64243 non-null  float64
 10  likes_rate       64243 non-null  float64
 11  comment_rate     64243 non-null  float64
 12  loc_tag          64243 non-null  float64
 13  hash_count       64243 non-null  float64
 14  cosine_sim_avg   64243 non-null  float64
 15  post_interval    64243 non-null  float64
 16  class            64243 non-null  object 
 17  posts_a     

In [6]:
account_t.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'posts', 'flw', 'flg', 'pic', 'link',
       'caption_len_avg', 'cap_zero_per', 'no_image_per', 'likes_rate',
       'comment_rate', 'loc_tag', 'hash_count', 'cosine_sim_avg',
       'post_interval', 'class', 'posts_a', 'flw_a', 'flg_a', 'likes_a',
       'hash_a', 'cap_avg_a', 'comment_r_a', 'post_interval_a'],
      dtype='object')

In [15]:
#Create dummy or indicator features from categorical features
#Split into testing and training datasets 

#dfo=account_t.select_dtypes(include=['object']) # to select object type columns
dfo=account_t[['pic','link']] # all data was numeric 
df_droped=account_t.drop(['Unnamed: 0', 'Unnamed: 0.1','pic','link', 'posts', 'flw', 'flg', 'likes_rate','hash_count', 'caption_len_avg', 'comment_rate', 'post_interval','cosine_sim_avg' ], axis=1)
df = pd.concat([df_droped, pd.get_dummies(dfo)], axis=1)
df.head()

,cap_zero_per,no_image_per,loc_tag,class,posts_a,flw_a,flg_a,likes_a,hash_a,cap_avg_a,comment_r_a,post_interval_a,pic,link
0,0.0,0.0,0.000,f,44.0,48.0,325.0,0.00,0.0,12.0,0.00,0.094985,1,0
1,0.0,1.0,0.000,f,10.0,66.0,321.0,14.39,1.5,213.0,1.97,230.412857,1,0
2,0.0,1.0,0.000,f,33.0,970.0,308.0,10.10,2.5,436.0,0.30,43.569939,1,1
3,1.0,0.0,0.000,f,70.0,86.0,360.0,0.78,0.0,0.0,0.06,5.859799,1,0
4,0.0,0.0,0.667,f,3.0,21.0,285.0,14.29,0.0,93.0,0.00,0.126019,1,0


In [14]:
# Check for missing data again
df.isnull().any()
#if there are missing values we'll use to fill in any missing values
#X_d_median = X_train.median()

Unnamed: 0         False
Unnamed: 0.1       False
cap_zero_per       False
no_image_per       False
loc_tag            False
class              False
posts_a            False
flw_a              False
flg_a              False
likes_a            False
hash_a             False
cap_avg_a          False
comment_r_a        False
post_interval_a    False
pic                False
link               False
dtype: bool

In [22]:
# split the dataset
X=df.drop(['class'], axis=1)
y=df['class']

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=24)

In [24]:
# Standardize the magnitude of numeric features using a scaler- since some of the features are percintage 0 to 1 
#and some are big numbers. 

scaler=StandardScaler()
scaler.fit(X_train)
X_train_s=scaler.transform(X_train)
X_test_s=scaler.transform(X_test)

In [ ]:
# Question 1- My categorical features were coded as int data type, is it good for the model?
            # do we need them to be int or object for the model to work ("correctly understanding the data")
# Question 2- There is no need to scale the categorical or the dummy variables but is it a mistake 
              #to transform the two i have (0/1) in the scaler